In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
df2=pd.read_csv('avocado.csv')
# --- PASO PREVIO: Asegurarse que 'df2' existe y está preparado ---
# Asumimos que df2 es el DataFrame original cargado previamente.
if 'df2' not in locals() or not isinstance(df2, pd.DataFrame):
    print("Error: El DataFrame 'df2' no parece estar cargado o no es un DataFrame.")
    exit()
else:
    print("Usando el DataFrame 'df2' existente.")
    # Verificar que la columna 'region' existe
    if 'region' not in df2.columns:
        print("Error: La columna 'region' no se encontró en df2.")
        exit()

# --- 1. Definir el Mapeo de Subregión a Región Principal ---

# !! IMPORTANTE !! Revisa y ajusta este diccionario según tu conocimiento
# o los criterios específicos que quieras usar para la jerarquía.
# He agrupado NNE bajo Northeast como sugeriste.
# Las 9 regiones principales se mapean a sí mismas.
# TotalUS se mapea a sí mismo.

region_mapping = {
    # Northeast (Incluyendo NorthernNewEngland)
    'Albany': 'Northeast', 'BaltimoreWashington': 'Northeast', 'Boston': 'Northeast',
    'BuffaloRochester': 'Northeast', 'HarrisburgScranton': 'Northeast',
    'HartfordSpringfield': 'Northeast', 'NewYork': 'Northeast', 'Philadelphia': 'Northeast',
    'Syracuse': 'Northeast', 'NorthernNewEngland': 'Northeast', # NNE agrupado en NE
    'Northeast': 'Northeast', # Región principal se mapea a sí misma

    # Southeast
    'Atlanta': 'Southeast', 'Charlotte': 'Southeast', 'Jacksonville': 'Southeast',
    'MiamiFtLauderdale': 'Southeast', 'Orlando': 'Southeast', 'RaleighGreensboro': 'Southeast',
    'RichmondNorfolk': 'Southeast', 'Roanoke': 'Southeast', 'SouthCarolina': 'Southeast',
    'Tampa': 'Southeast', 'Nashville': 'Southeast', # Nashville más común en SE
    'Southeast': 'Southeast', # Región principal

    # GreatLakes (Incluye algunas ciudades que a veces se ponen en Midsouth)
    'Chicago': 'GreatLakes', 'CincinnatiDayton': 'GreatLakes', 'Columbus': 'GreatLakes',
    'Detroit': 'GreatLakes', 'GrandRapids': 'GreatLakes', 'Indianapolis': 'GreatLakes',
    'Pittsburgh': 'GreatLakes',
    'GreatLakes': 'GreatLakes', # Región principal

    # Midsouth (Según tu lista de principales)
    'Louisville': 'Midsouth', 'StLouis': 'Midsouth', # StLouis aquí
    'Midsouth': 'Midsouth', # Región principal

    # SouthCentral
    'DallasFtWorth': 'SouthCentral', 'Houston': 'SouthCentral', 'NewOrleansMobile': 'SouthCentral',
    'SouthCentral': 'SouthCentral', # Región principal

    # WestTexNewMexico
    'WestTexNewMexico': 'WestTexNewMexico', # Región principal

    # Plains
    'Boise': 'Plains', 'Denver': 'Plains', # Boise asignado aquí
    'Plains': 'Plains', # Región principal

    # West
    'California': 'West', 'LasVegas': 'West', 'LosAngeles': 'West', 'PhoenixTucson': 'West',
    'Portland': 'West', 'Sacramento': 'West', 'SanDiego': 'West', 'SanFrancisco': 'West',
    'Seattle': 'West', 'Spokane': 'West',
    'West': 'West', # Región principal

    # Caso Especial
    'TotalUS': 'TotalUS'
}

print("\nMapeo de Regiones Definido.")
# Puedes imprimir el diccionario 'region_mapping' si quieres verlo completo

# --- 2. Crear la Nueva Columna "Clasificación" ---

# Usamos el método .map() en la columna 'region' de df2.
# .map() buscará cada valor de 'region' en las claves del diccionario 'region_mapping'
# y devolverá el valor asociado (la región principal).
# Si una región de df2 no está en el diccionario, .map() devolverá NaN.
df2['Clasificación'] = df2['region'].map(region_mapping)

print("Nueva columna 'Clasificación' creada.")

# --- 3. Verificar Resultados ---

# Contar cuántas filas corresponden a cada Región Principal (Clasificación)
print("\nConteo de filas por nueva 'Clasificación':")
print(df2['Clasificación'].value_counts(dropna=False)) # dropna=False muestra si hay NaNs (regiones no mapeadas)

# Verificar si hubo regiones que no se pudieron mapear (resultando en NaN)
unmapped_regions = df2[df2['Clasificación'].isnull()]['region'].unique()
if len(unmapped_regions) > 0:
    print(f"\n¡Advertencia! Las siguientes regiones en df2 no se encontraron en el mapeo y tienen 'Clasificación' = NaN:")
    print(unmapped_regions)
    print("Deberías añadirlas al diccionario 'region_mapping' si quieres clasificarlas.")
else:
    print("\nTodas las regiones fueron mapeadas exitosamente.")

# Mostrar las primeras filas con la nueva columna
print("\nPrimeras 5 filas de df2 con la nueva columna 'Clasificación':")
print(df2[['region', 'Clasificación']].head())

# Mostrar un ejemplo de una subregión y su clasificación
print("\nEjemplo de clasificación para 'LosAngeles':")
print(df2[df2['region'] == 'LosAngeles'][['region', 'Clasificación']].head(1))
print("\nEjemplo de clasificación para 'West' (región principal):")
print(df2[df2['region'] == 'West'][['region', 'Clasificación']].head(1))

Usando el DataFrame 'df2' existente.

Mapeo de Regiones Definido.
Nueva columna 'Clasificación' creada.

Conteo de filas por nueva 'Clasificación':
Clasificación
Southeast           4056
Northeast           3718
West                3718
GreatLakes          2704
SouthCentral        1352
Plains              1014
Midsouth            1014
TotalUS              338
WestTexNewMexico     335
Name: count, dtype: int64

Todas las regiones fueron mapeadas exitosamente.

Primeras 5 filas de df2 con la nueva columna 'Clasificación':
   region Clasificación
0  Albany     Northeast
1  Albany     Northeast
2  Albany     Northeast
3  Albany     Northeast
4  Albany     Northeast

Ejemplo de clasificación para 'LosAngeles':
          region Clasificación
1144  LosAngeles          West

Ejemplo de clasificación para 'West' (región principal):
     region Clasificación
2704   West          West


## KNN

In [3]:

# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import datetime
import gc

# Modelos, preprocesamiento y métricas de Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# --- Configuración ---
TEST_SIZE = 0.20
RANDOM_STATE = 42
MIN_SAMPLES_FOR_EVAL = 10  # Mínimo de muestras NO CERO para intentar evaluación
MIN_SAMPLES_FOR_TRAIN = 5  # Mínimo de muestras NO CERO para entrenar modelo final (debe ser >= K)
N_NEIGHBORS_KNN = 5
R2_THRESHOLD = 0.69      # Umbral R² para decidir si imputar
PROPORTION_ZEROS_THRESHOLD = 0.50 # Umbral máximo de ceros permitido

COLUMNA_TARGET = 'Large Bags'
COLUMNA_PREDICCION = 'KNN_Prediction' # Columna donde guardar predicciones

# --- Preparación de df2 (Asegúrate que esta sección esté completa y funcione) ---
# [COPIAR AQUÍ LA SECCIÓN DE PREPARACIÓN DE df2 DEL SCRIPT ANTERIOR]
# --- Inicio de sección copiada ---
if 'df2' not in locals() or not isinstance(df2, pd.DataFrame):
    print("Error: El DataFrame 'df2' no está definido. Cárgalo primero."); exit()
else:
    print("Usando el DataFrame 'df2' existente.")

print("\nPreparando datos en df2 (índice, tipos numéricos, features de tiempo)...")
try:
    # Asegurar índice Date
    if not isinstance(df2.index, pd.DatetimeIndex):
        if 'Date' in df2.columns:
             df2['Date'] = pd.to_datetime(df2['Date'])
             df2.set_index('Date', inplace=True)
             df2.sort_index(inplace=True)
        elif not isinstance(df2.index, pd.DatetimeIndex):
             df2.index = pd.to_datetime(df2.index)
             df2.sort_index(inplace=True)
        else:
             df2.sort_index(inplace=True)
        print("  - Índice Datetime preparado y ordenado.")
    elif not df2.index.is_monotonic_increasing:
        df2.sort_index(inplace=True)
        print("  - Índice Datetime ordenado.")

    # Crear Features de Tiempo para KNN
    features = ['Fecha_Ordinal', 'Mes', 'SemanaDelAno'] # Definir features aquí
    if 'Fecha_Ordinal' not in df2.columns:
        df2['Fecha_Ordinal'] = df2.index.map(datetime.date.toordinal)
        print("  - Creada columna 'Fecha_Ordinal'.")
    if 'Mes' not in df2.columns:
        df2['Mes'] = df2.index.month
        print("  - Creada columna 'Mes'.")
    if 'SemanaDelAno' not in df2.columns:
        df2['SemanaDelAno'] = df2.index.isocalendar().week.astype(np.int32)
        print("  - Creada columna 'SemanaDelAno'.")
except Exception as e:
    print(f"Error durante la preparación de datos: {e}"); exit()

cols_to_check = ['Large Bags', 'Total Bags', 'Total Volume', 'AveragePrice']
print(f"Asegurando que {cols_to_check} sean numéricas y rellenando NaNs con 0...")
for col in cols_to_check:
    if col in df2.columns:
         if not pd.api.types.is_numeric_dtype(df2[col]):
              print(f"  - Convirtiendo '{col}' a numérico...")
              df2[col] = pd.to_numeric(df2[col], errors='coerce')
         if df2[col].isnull().any():
              print(f"  - Rellenando NaNs en '{col}' con 0...")
              df2[col] = df2[col].fillna(0)

print("Optimizando tipos de datos...")
try:
    if 'region' in df2.columns: df2['region'] = df2['region'].astype('category')
    if 'type' in df2.columns: df2['type'] = df2['type'].astype('category')
    for col in df2.select_dtypes(include=['float64']).columns:
        # Evitar convertir KNN_Prediction si ya existe y tiene NaNs
        if col != COLUMNA_PREDICCION or not (COLUMNA_PREDICCION in df2 and df2[col].isnull().any()):
            df2[col] = df2[col].astype(np.float32)
    for col in df2.select_dtypes(include=['int64']).columns:
        if df2[col].min() >= np.iinfo(np.int32).min and df2[col].max() <= np.iinfo(np.int32).max:
             df2[col] = df2[col].astype(np.int32)
    print("Optimización de tipos completada.")
except Exception as e_opt: print(f"Advertencia durante optimización de tipos: {e_opt}")
# --- Fin de sección copiada ---

# --- Evaluación y posterior Imputación Condicional con KNN ---
print(f"\n--- Iniciando Evaluación e Imputación Condicional (si R² > {R2_THRESHOLD}) para '{COLUMNA_TARGET}' ---")

# Inicializar columna para guardar predicciones (solo para los ceros imputados)
df2[COLUMNA_PREDICCION] = np.nan

# Lista para almacenar resultados de evaluación
evaluation_results = []
imputed_points_total = 0

# Asegurar columnas antes del bucle
required_loop_cols = ['type', 'region', COLUMNA_TARGET] + features
if not all(col in df2.columns for col in required_loop_cols):
      print(f"Error: Faltan columnas necesarias ({required_loop_cols}) en df2."); exit()

# --- Bucle Principal por Grupo ---
total_grupos = len(df2.groupby(['type', 'region']))
grupo_actual = 0

for (tipo, region), group_df in df2.groupby(['type', 'region']):
    grupo_actual += 1
    print(f"\nProcesando Grupo {grupo_actual}/{total_grupos}: {tipo}/{region}...")

    # Variables para limpieza en finally
    # Evaluación
    knn_eval = None; scaler_eval = None; y_pred_eval = None; X_eval = None; y_eval = None;
    X_train_eval = None; X_test_eval = None; y_train_eval = None; y_test_eval = None;
    X_train_scaled_eval = None; X_test_scaled_eval = None; non_zero_data = None;
    # Imputación
    knn_impute = None; scaler_impute = None; X_all_train = None; y_all_train = None;
    X_impute_features = None; X_all_train_scaled = None; X_impute_scaled = None;
    imputed_values = None; zero_data = None

    r2_eval = np.nan # Inicializar R² de evaluación como NaN

    try:
        # 1. Separar datos cero y no cero
        non_zero_data = group_df[group_df[COLUMNA_TARGET] > 0].copy()
        zero_data = group_df[group_df[COLUMNA_TARGET] == 0].copy()
        n_non_zero = len(non_zero_data)
        n_zero = len(zero_data)
        n_total = len(group_df)

        print(f"  - Total: {n_total}, No Cero: {n_non_zero}, Cero: {n_zero}")

        # 2. Chequeos iniciales (antes de evaluar o imputar)
        if n_zero == 0:
            print("  - OK: No hay ceros que imputar en este grupo.")
            continue # Pasar al siguiente grupo

        proportion_zeros = n_zero / n_total if n_total > 0 else 0
        if proportion_zeros >= PROPORTION_ZEROS_THRESHOLD:
            print(f"  - OMITIDO IMPUTACIÓN: Proporción de ceros >= {PROPORTION_ZEROS_THRESHOLD*100:.0f}% ({proportion_zeros*100:.1f}%).")
            continue

        if n_non_zero < MIN_SAMPLES_FOR_TRAIN: # Mínimo absoluto para entrenar algo
             print(f"  - OMITIDO IMPUTACIÓN: Insuficientes muestras no cero ({n_non_zero} < {MIN_SAMPLES_FOR_TRAIN}) para entrenar.")
             continue

        # --- 3. Fase de Evaluación (si hay suficientes datos) ---
        print("  - Fase de Evaluación:")
        if n_non_zero < MIN_SAMPLES_FOR_EVAL:
            print(f"    - OMITIDA: Insuficientes muestras no cero ({n_non_zero} < {MIN_SAMPLES_FOR_EVAL}) para una evaluación fiable.")
            r2_eval = np.nan # Marcar que no se pudo evaluar
        else:
            # Preparar X e y para evaluación
            X_eval = non_zero_data[features]
            y_eval = non_zero_data[COLUMNA_TARGET]

            # Dividir
            X_train_eval, X_test_eval, y_train_eval, y_test_eval = train_test_split(
                X_eval, y_eval, test_size=TEST_SIZE, random_state=RANDOM_STATE
            )
            print(f"    - Split Evaluación: {len(X_train_eval)} train / {len(X_test_eval)} test")

            # Escalar (basado en train_eval)
            scaler_eval = StandardScaler()
            X_train_scaled_eval = scaler_eval.fit_transform(X_train_eval)
            X_test_scaled_eval = scaler_eval.transform(X_test_eval)

            # Entrenar modelo de evaluación
            knn_eval = KNeighborsRegressor(n_neighbors=N_NEIGHBORS_KNN, weights='distance')
            knn_eval.fit(X_train_scaled_eval, y_train_eval)

            # Predecir y Evaluar en test_eval
            y_pred_eval = knn_eval.predict(X_test_scaled_eval)
            y_pred_eval = np.maximum(0, y_pred_eval) # Ajustar negativos

            mae_eval = mean_absolute_error(y_test_eval, y_pred_eval)
            rmse_eval = np.sqrt(mean_squared_error(y_test_eval, y_pred_eval))
            r2_eval = knn_eval.score(X_test_scaled_eval, y_test_eval) # R²

            print(f"    - Resultados Evaluación (Test Set): MAE={mae_eval:,.2f}, RMSE={rmse_eval:,.2f}, R²={r2_eval:.4f}")

            # Guardar resultados de evaluación
            evaluation_results.append({
                'type': tipo, 'region': region, 'n_non_zero': n_non_zero, 'n_train_eval': len(X_train_eval),
                'n_test_eval': len(X_test_eval), 'MAE_eval': mae_eval, 'RMSE_eval': rmse_eval, 'R2_eval': r2_eval
            })

        # --- 4. Fase de Imputación Condicional (si R² > umbral) ---
        print("  - Fase de Imputación:")
        # Proceder solo si la evaluación fue posible Y el R² supera el umbral
        if not np.isnan(r2_eval) and r2_eval > R2_THRESHOLD:
            print(f"    - PROCEDIENDO: R² ({r2_eval:.4f}) > umbral ({R2_THRESHOLD}).")
            print(f"    - Re-entrenando KNN con TODOS los {n_non_zero} datos no cero...")

            # Preparar datos para el modelo FINAL de imputación
            X_all_train = non_zero_data[features]
            y_all_train = non_zero_data[COLUMNA_TARGET]
            X_impute_features = zero_data[features] # Features de los datos a imputar

            # Escalar: Ajustar con TODOS los datos no cero
            scaler_impute = StandardScaler()
            X_all_train_scaled = scaler_impute.fit_transform(X_all_train)
            # Aplicar mismo escalador a los datos a imputar
            X_impute_scaled = scaler_impute.transform(X_impute_features)

            # Entrenar modelo FINAL de imputación
            knn_impute = KNeighborsRegressor(n_neighbors=N_NEIGHBORS_KNN, weights='distance')
            knn_impute.fit(X_all_train_scaled, y_all_train)

            # Predecir los valores para los ceros
            imputed_values = knn_impute.predict(X_impute_scaled)
            imputed_values = np.maximum(0, imputed_values) # Ajustar negativos

            # Guardar las predicciones en la columna COLUMNA_PREDICCION
            indices_imputar = zero_data.index
            df2.loc[indices_imputar, COLUMNA_PREDICCION] = imputed_values
            imputed_points_total += len(indices_imputar)
            print(f"    - ¡ÉXITO! Se guardaron {len(indices_imputar)} predicciones en '{COLUMNA_PREDICCION}'.")

        elif np.isnan(r2_eval):
             print(f"    - OMITIDA: No se pudo realizar la evaluación previa (pocos datos).")
        else:
             print(f"    - OMITIDA: R² ({r2_eval:.4f}) no supera el umbral ({R2_THRESHOLD}).")


    except Exception as e:
        print(f"  - ERROR Inesperado ({type(e).__name__}: {str(e)[:100]}...).")

    finally:
         # Limpieza de Memoria
         variables_a_borrar = [
             'group_df', 'non_zero_data', 'zero_data',
             'knn_eval', 'scaler_eval', 'y_pred_eval', 'X_eval', 'y_eval', 'X_train_eval',
             'X_test_eval', 'y_train_eval', 'y_test_eval', 'X_train_scaled_eval', 'X_test_scaled_eval',
             'knn_impute', 'scaler_impute', 'X_all_train', 'y_all_train', 'X_impute_features',
             'X_all_train_scaled', 'X_impute_scaled', 'imputed_values'
             ]
         for var_name in variables_a_borrar:
              try:
                  if var_name in locals(): del locals()[var_name]
              except NameError: pass
         # gc.collect()

print(f"\n--- Proceso de Evaluación e Imputación Condicional Finalizado ---")
print(f"Se guardaron predicciones en '{COLUMNA_PREDICCION}' para un total de {imputed_points_total} puntos (ceros originales).")
print(f"Para los demás ceros, o grupos omitidos, '{COLUMNA_PREDICCION}' contendrá NaN.")

# --- Opcional: Mostrar Resultados Consolidados de la Evaluación ---
if evaluation_results:
    results_eval_df = pd.DataFrame(evaluation_results)
    print("\nResumen de Resultados de la Fase de Evaluación (Solo grupos evaluados):")
    pd.set_option('display.max_rows', None); pd.set_option('display.max_columns', None); pd.set_option('display.width', 120)
    print(results_eval_df.sort_values(by='R2_eval', ascending=False).round({'MAE_eval': 2, 'RMSE_eval': 2, 'R2_eval': 4}))
    pd.reset_option('display.max_rows'); pd.reset_option('display.max_columns'); pd.reset_option('display.width')
else:
    print("\nNo se generaron resultados de evaluación (quizás por falta de datos suficientes en los grupos).")


Usando el DataFrame 'df2' existente.

Preparando datos en df2 (índice, tipos numéricos, features de tiempo)...
  - Índice Datetime preparado y ordenado.
  - Creada columna 'Fecha_Ordinal'.
  - Creada columna 'Mes'.
  - Creada columna 'SemanaDelAno'.
Asegurando que ['Large Bags', 'Total Bags', 'Total Volume', 'AveragePrice'] sean numéricas y rellenando NaNs con 0...
Optimizando tipos de datos...
Optimización de tipos completada.

--- Iniciando Evaluación e Imputación Condicional (si R² > 0.69) para 'Large Bags' ---

Procesando Grupo 1/108: conventional/Albany...
  - Total: 169, No Cero: 169, Cero: 0
  - OK: No hay ceros que imputar en este grupo.

Procesando Grupo 2/108: conventional/Atlanta...
  - Total: 169, No Cero: 169, Cero: 0
  - OK: No hay ceros que imputar en este grupo.

Procesando Grupo 3/108: conventional/BaltimoreWashington...
  - Total: 169, No Cero: 169, Cero: 0
  - OK: No hay ceros que imputar en este grupo.

Procesando Grupo 4/108: conventional/Boise...
  - Total: 169, N

<ipython-input-3-dd99cda5a580>:107: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_grupos = len(df2.groupby(['type', 'region']))
<ipython-input-3-dd99cda5a580>:110: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (tipo, region), group_df in df2.groupby(['type', 'region']):


    - Resultados Evaluación (Test Set): MAE=1,555.34, RMSE=2,749.31, R²=0.7981
  - Fase de Imputación:
    - PROCEDIENDO: R² (0.7981) > umbral (0.69).
    - Re-entrenando KNN con TODOS los 122 datos no cero...
  - ERROR Inesperado (ValueError: Must have equal len keys and value when setting with an iterable...).

Procesando Grupo 49/108: conventional/StLouis...
  - Total: 169, No Cero: 153, Cero: 16
  - Fase de Evaluación:
    - Split Evaluación: 122 train / 31 test
    - Resultados Evaluación (Test Set): MAE=4,565.90, RMSE=6,237.72, R²=0.6260
  - Fase de Imputación:
    - OMITIDA: R² (0.6260) no supera el umbral (0.69).

Procesando Grupo 50/108: conventional/Syracuse...
  - Total: 169, No Cero: 113, Cero: 56
  - Fase de Evaluación:
    - Split Evaluación: 90 train / 23 test
    - Resultados Evaluación (Test Set): MAE=1,363.87, RMSE=2,871.12, R²=0.7855
  - Fase de Imputación:
    - PROCEDIENDO: R² (0.7855) > umbral (0.69).
    - Re-entrenando KNN con TODOS los 113 datos no cero...
  - 

In [4]:

# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import datetime
import gc # Para limpieza de memoria

# Modelos, preprocesamiento y métricas de Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ¡¡NUEVO!! Importaciones para gráficos
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# --- Configuración ---
TEST_SIZE = 0.20
RANDOM_STATE = 42
MIN_SAMPLES_FOR_EVAL = 10
N_NEIGHBORS_KNN = 5

# --- Preparación de df2 (Asegúrate que esta sección esté completa y funcione) ---
# [COPIAR AQUÍ LA SECCIÓN DE PREPARACIÓN DE df2 DEL SCRIPT ANTERIOR]
# --- Inicio de sección copiada ---
if 'df2' not in locals() or not isinstance(df2, pd.DataFrame):
    print("Error: El DataFrame 'df2' no está definido. Cárgalo primero."); exit()
else:
    print("Usando el DataFrame 'df2' existente.")

print("\nPreparando datos en df2 (índice, tipos numéricos, features de tiempo)...")
try:
    # Asegurar índice Date
    if not isinstance(df2.index, pd.DatetimeIndex):
        if 'Date' in df2.columns:
             df2['Date'] = pd.to_datetime(df2['Date'])
             df2.set_index('Date', inplace=True)
             df2.sort_index(inplace=True)
        elif not isinstance(df2.index, pd.DatetimeIndex):
             df2.index = pd.to_datetime(df2.index)
             df2.sort_index(inplace=True)
        else:
             df2.sort_index(inplace=True)
        print("  - Índice Datetime preparado y ordenado.")
    elif not df2.index.is_monotonic_increasing:
        df2.sort_index(inplace=True)
        print("  - Índice Datetime ordenado.")

    # Crear Features de Tiempo para KNN
    if 'Fecha_Ordinal' not in df2.columns:
        df2['Fecha_Ordinal'] = df2.index.map(datetime.date.toordinal)
        print("  - Creada columna 'Fecha_Ordinal'.")
    if 'Mes' not in df2.columns:
        df2['Mes'] = df2.index.month
        print("  - Creada columna 'Mes'.")
    if 'SemanaDelAno' not in df2.columns:
        df2['SemanaDelAno'] = df2.index.isocalendar().week.astype(np.int32)
        print("  - Creada columna 'SemanaDelAno'.")
except Exception as e:
    print(f"Error durante la preparación de datos: {e}"); exit()

cols_to_check = ['Large Bags', 'Total Bags', 'Total Volume', 'AveragePrice']
print(f"Asegurando que {cols_to_check} sean numéricas y rellenando NaNs con 0...")
for col in cols_to_check:
    if col in df2.columns:
         if not pd.api.types.is_numeric_dtype(df2[col]):
              print(f"  - Convirtiendo '{col}' a numérico...")
              df2[col] = pd.to_numeric(df2[col], errors='coerce')
         if df2[col].isnull().any():
              print(f"  - Rellenando NaNs en '{col}' con 0...")
              df2[col] = df2[col].fillna(0)

print("Optimizando tipos de datos...")
try:
    if 'region' in df2.columns: df2['region'] = df2['region'].astype('category')
    if 'type' in df2.columns: df2['type'] = df2['type'].astype('category')
    for col in df2.select_dtypes(include=['float64']).columns:
       # Evitar convertir la columna de predicción si ya existe (aunque no se usará aquí)
       # if col != 'KNN_Prediction' or not ('KNN_Prediction' in df2 and df2[col].isnull().any()):
          df2[col] = df2[col].astype(np.float32)
    for col in df2.select_dtypes(include=['int64']).columns:
        if df2[col].min() >= np.iinfo(np.int32).min and df2[col].max() <= np.iinfo(np.int32).max:
             df2[col] = df2[col].astype(np.int32)
    print("Optimización de tipos completada.")
except Exception as e_opt: print(f"Advertencia durante optimización de tipos: {e_opt}")
# --- Fin de sección copiada ---


# --- Evaluación del Modelo KNN por Grupo con Split Train/Test ---
print(f"\n--- Iniciando Evaluación de KNN para 'Large Bags' (Split {((1-TEST_SIZE)*100):.0f}% Entrenamiento / {TEST_SIZE*100:.0f}% Prueba) ---")

evaluation_results = []
required_loop_cols = ['type', 'region', 'Large Bags', 'Fecha_Ordinal', 'Mes', 'SemanaDelAno']
if not all(col in df2.columns for col in required_loop_cols):
      print(f"Error: Faltan columnas necesarias ({required_loop_cols}) en df2."); exit()

total_grupos = len(df2.groupby(['type', 'region']))
grupo_actual = 0

for (tipo, region), group_df in df2.groupby(['type', 'region']):
    grupo_actual += 1
    print(f"\nProcesando Grupo {grupo_actual}/{total_grupos}: {tipo}/{region}...")

    # Variables para limpieza
    knn_reg = None; scaler = None; y_pred = None; X = None; y = None; X_train = None; X_test = None; y_train = None; y_test = None
    X_train_scaled = None; X_test_scaled = None; non_zero_data = None; r2 = np.nan # Inicializar r2

    try:
        # 1. Seleccionar datos no cero
        non_zero_data = group_df[group_df['Large Bags'] > 0].copy()
        n_non_zero = len(non_zero_data)
        print(f"  - Muestras con 'Large Bags' > 0: {n_non_zero}")

        # 2. Verificar suficientes datos
        if n_non_zero < MIN_SAMPLES_FOR_EVAL:
            print(f"  - OMITIDO: Insuficientes muestras no cero ({n_non_zero} < {MIN_SAMPLES_FOR_EVAL}) para evaluar.")
            continue

        # 3. Preparar Features (X) y Target (y)
        features = ['Fecha_Ordinal', 'Mes', 'SemanaDelAno']
        X = non_zero_data[features]
        y = non_zero_data['Large Bags']

        # 4. Dividir en Entrenamiento y Prueba
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
        )
        print(f"  - Split: {len(X_train)} entrenamiento / {len(X_test)} prueba")

        # 5. Escalar Características
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # 6. Entrenar KNN
        knn_reg = KNeighborsRegressor(n_neighbors=N_NEIGHBORS_KNN, weights='distance')
        knn_reg.fit(X_train_scaled, y_train)
        print(f"  - Modelo KNN entrenado (k={N_NEIGHBORS_KNN}).")

        # 7. Predecir en Prueba
        y_pred = knn_reg.predict(X_test_scaled)
        y_pred = np.maximum(0, y_pred) # Ajustar negativos

        # 8. Evaluar
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = knn_reg.score(X_test_scaled, y_test) # Usar .score() para R²

        print(f"  - EVALUACIÓN (Prueba):")
        print(f"    - MAE : {mae:,.2f}")
        print(f"    - RMSE: {rmse:,.2f}")
        print(f"    - R²  : {r2:.4f}")

        # Guardar resultados
        evaluation_results.append({
            'type': tipo, 'region': region, 'n_non_zero': n_non_zero, 'n_train': len(X_train),
            'n_test': len(X_test), 'MAE': mae, 'RMSE': rmse, 'R2': r2
        })

        # --- 9. NUEVO: Graficar Comparación Actual vs Predicción ---
        # Solo intentar graficar si hay datos de prueba
        if not X_test.empty:
            print(f"  - Generando gráfico de dispersión Actual vs. Predicción...")
            try:
                plt.figure(figsize=(14, 7)) # Nueva figura para este grupo

                # Graficar valores reales del conjunto de prueba (azules)
                plt.scatter(X_test.index, y_test,
                            color='blue', label='Valor Real (Prueba)', alpha=0.6, s=50, marker='o')

                # Graficar predicciones para el conjunto de prueba (naranjas)
                plt.scatter(X_test.index, y_pred,
                            color='orange', label='Predicción KNN', marker='x', s=50)

                # Título y etiquetas
                plt.title(f"Comparación Real vs. Predicción KNN - Large Bags\n{tipo.capitalize()} / {region} (Test Set, R²={r2:.3f})", fontsize=14)
                plt.xlabel("Fecha", fontsize=12)
                plt.ylabel("Large Bags", fontsize=12)
                plt.legend()
                plt.grid(True, axis='y', linestyle=':') # Rejilla horizontal suave

                # Formatear eje de fechas para mejor lectura
                plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d')) # Formato más específico
                plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator(minticks=5, maxticks=10)) # Ajustar número de ticks
                plt.gcf().autofmt_xdate() # Rotar/ajustar fechas automáticamente

                plt.tight_layout() # Ajustar para evitar solapamiento de etiquetas
                plt.show() # Mostrar el gráfico para este grupo

            except Exception as e_plot:
                print(f"  - ERROR al generar gráfico ({type(e_plot).__name__}: {e_plot}).")
        else:
             print("  - No hay datos de prueba para graficar.")


    except Exception as e:
        print(f"  - ERROR durante la evaluación ({type(e).__name__}: {str(e)[:100]}...).")

    finally:
         # Limpieza de Memoria
         variables_a_borrar = ['group_df', 'non_zero_data', 'X', 'y',
                               'X_train', 'X_test', 'y_train', 'y_test',
                               'X_train_scaled', 'X_test_scaled', 'knn_reg', 'scaler', 'y_pred']
         for var_name in variables_a_borrar:
              try:
                  if var_name in locals(): del locals()[var_name]
              except NameError: pass
         # gc.collect()

print(f"\n--- Evaluación de KNN finalizada para todos los grupos ---")

# --- Opcional: Mostrar Resultados Consolidados ---
if evaluation_results:
    results_df = pd.DataFrame(evaluation_results)
    print("\nResumen de Resultados de Evaluación (Ordenado por R² descendente):")
    pd.set_option('display.max_rows', None); pd.set_option('display.max_columns', None); pd.set_option('display.width', 120)
    print(results_df.sort_values(by='R2', ascending=False).round({'MAE': 2, 'RMSE': 2, 'R2': 4}))
    pd.reset_option('display.max_rows'); pd.reset_option('display.max_columns'); pd.reset_option('display.width')
else:
    print("\nNo se generaron resultados de evaluación.")


Output hidden; open in https://colab.research.google.com to view.

In [5]:

# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
# ¡Importante! Modelos y preprocesamiento de Scikit-learn
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler # Para escalar características
import io
import datetime
import gc # Para limpieza de memoria

# --- Definir nombre para la nueva columna de predicciones ---
COLUMNA_PREDICCION = 'KNN_Prediction'

# --- Preparación ---
# **¡ASEGÚRATE DE QUE df2 CONTENGA TUS DATOS ANTES DE EJECUTAR ESTO!**
if 'df2' not in locals() or not isinstance(df2, pd.DataFrame):
    print("Error: El DataFrame 'df2' no está definido. Cárgalo primero."); exit()
else:
    print("Usando el DataFrame 'df2' existente.")

# --- Preparación de Datos en df2 ---
print("\nPreparando datos en df2 (índice, tipos numéricos, features de tiempo)...")
try:
    # Asegurar índice Date
    if not isinstance(df2.index, pd.DatetimeIndex):
        if 'Date' in df2.columns:
             # Convertir Date a datetime, establecer como índice y ordenar
             df2['Date'] = pd.to_datetime(df2['Date'])
             df2.set_index('Date', inplace=True)
             df2.sort_index(inplace=True)
        # Si 'Date' no es columna pero el índice no es DatetimeIndex, intentar convertir índice
        elif not isinstance(df2.index, pd.DatetimeIndex):
             df2.index = pd.to_datetime(df2.index)
             df2.sort_index(inplace=True)
        else: # Si ya es DatetimeIndex pero no está ordenado
             df2.sort_index(inplace=True)
        print("  - Índice Datetime preparado y ordenado.")
    elif not df2.index.is_monotonic_increasing: # Si es DatetimeIndex pero no ordenado
        df2.sort_index(inplace=True)
        print("  - Índice Datetime ordenado.")


    # *** NUEVO: Crear Features de Tiempo para KNN ***
    # Necesitamos características numéricas que capturen la tendencia y estacionalidad
    if 'Fecha_Ordinal' not in df2.columns:
        df2['Fecha_Ordinal'] = df2.index.map(datetime.date.toordinal)
        print("  - Creada columna 'Fecha_Ordinal'.")
    # Usaremos Semana del Año y Mes para capturar estacionalidad
    if 'Mes' not in df2.columns:
        df2['Mes'] = df2.index.month
        print("  - Creada columna 'Mes'.")
    if 'SemanaDelAno' not in df2.columns:
        df2['SemanaDelAno'] = df2.index.isocalendar().week.astype(np.int32) # Usar tipo más eficiente
        print("  - Creada columna 'SemanaDelAno'.")

except Exception as e:
    print(f"Error durante la preparación de datos: {e}"); exit()

# Asegurar columnas numéricas clave y rellenar NaNs con 0
cols_to_check = ['Large Bags', 'Total Bags', 'Total Volume', 'AveragePrice'] # Añadir AveragePrice por si acaso
print(f"Asegurando que {cols_to_check} sean numéricas y rellenando NaNs con 0...")
for col in cols_to_check:
    if col in df2.columns:
         if not pd.api.types.is_numeric_dtype(df2[col]):
              print(f"  - Convirtiendo '{col}' a numérico...")
              df2[col] = pd.to_numeric(df2[col], errors='coerce')
         if df2[col].isnull().any():
              print(f"  - Rellenando NaNs en '{col}' con 0...")
              df2[col] = df2[col].fillna(0)
    # else: # No imprimir advertencia si no existen
         # print(f"Advertencia: La columna '{col}' no se encontró en df2.")

# Optimización de tipos (opcional pero recomendado)
print("Optimizando tipos de datos...")
try:
    if 'region' in df2.columns: df2['region'] = df2['region'].astype('category')
    if 'type' in df2.columns: df2['type'] = df2['type'].astype('category')
    # Convertir floats a float32
    for col in df2.select_dtypes(include=['float64']).columns:
        # Evitar convertir la columna de predicción si ya existe y es float64 con NaNs
        if col != COLUMNA_PREDICCION or not df2[col].isnull().any():
           df2[col] = df2[col].astype(np.float32)
    # Convertir ints a int32 si caben
    for col in df2.select_dtypes(include=['int64']).columns:
        if df2[col].min() >= np.iinfo(np.int32).min and df2[col].max() <= np.iinfo(np.int32).max:
             df2[col] = df2[col].astype(np.int32)
    print("Optimización de tipos completada.")
except Exception as e_opt: print(f"Advertencia durante optimización de tipos: {e_opt}")


# --- Imputación por Grupo con KNeighborsRegressor en df2 ---
print("\n--- Iniciando proceso de imputación para 'Large Bags' con KNeighborsRegressor en df2 ---")
print(f"--- Las predicciones se guardarán en la columna '{COLUMNA_PREDICCION}' ---")

# *** NUEVO: Inicializar la columna de predicciones con NaN ***
df2[COLUMNA_PREDICCION] = np.nan

# KNN necesita algunos vecinos, pongamos un mínimo razonable
min_train_samples = 10 # Número mínimo de filas con Large Bags > 0 para entrenar KNN
print(f"Umbral mínimo de muestras de entrenamiento (>0): {min_train_samples}")
n_neighbors_knn = 5 # Número de vecinos a considerar por KNN
print(f"Número de vecinos (k) para KNN: {n_neighbors_knn}")
indices_imputados_global = set() # Mantenemos registro de qué índices recibieron predicción

# Asegurar columnas antes del bucle
required_loop_cols = ['type', 'region', 'Large Bags', 'Fecha_Ordinal', 'Mes', 'SemanaDelAno']
if not all(col in df2.columns for col in required_loop_cols):
      print(f"Error: Faltan columnas necesarias ({required_loop_cols}) en df2."); exit()

# --- Bucle Principal por Grupo ---
total_grupos = len(df2.groupby(['type', 'region']))
grupo_actual = 0
for (tipo, region), group_df in df2.groupby(['type', 'region']):
    grupo_actual += 1
    print(f"Procesando Grupo {grupo_actual}/{total_grupos}: {tipo}/{region}...", end=' ')

    # Definir variables que podrían crearse dentro del try para borrar en finally
    knn_reg = None; scaler = None; predicciones = None; predicciones_ajustadas = None
    X_train = None; y_train = None; X_impute = None; X_train_scaled = None; X_impute_scaled = None
    datos_entrenamiento = None; datos_imputar = None

    try:
        total_rows_group = len(group_df)
        if total_rows_group == 0: continue # Saltar si el grupo está vacío por alguna razón

        # Separar datos entrenamiento (>0) e imputación (==0)
        mask_train = group_df['Large Bags'] > 0
        mask_impute = group_df['Large Bags'] == 0
        datos_entrenamiento = group_df.loc[mask_train]
        datos_imputar = group_df.loc[mask_impute]
        zero_rows_group = len(datos_imputar)

        # Chequeo >= 50% ceros
        proportion_zeros = zero_rows_group / total_rows_group if total_rows_group > 0 else 0
        if proportion_zeros >= 0.5:
             print(f" Omitido ({proportion_zeros*100:.1f}% ceros)."); continue

        # Chequeo datos entrenamiento suficientes para KNN (necesita al menos k vecinos)
        if len(datos_entrenamiento) < min_train_samples or len(datos_entrenamiento) < n_neighbors_knn:
             print(f" Omitido (Insuf. Train: {len(datos_entrenamiento)} < {max(min_train_samples, n_neighbors_knn)})."); continue

        # Chequeo si hay algo que imputar (aunque no supere el 50% de ceros)
        if datos_imputar.empty:
             print(" OK (No hay ceros que imputar)."); continue

        print(f" Entrenando con {len(datos_entrenamiento)}, prediciendo para {zero_rows_group} ceros...", end=' ')

        # Preparar Features (X) y Target (y)
        # Usamos Fecha_Ordinal (tendencia), Mes y SemanaDelAno (estacionalidad)
        features = ['Fecha_Ordinal', 'Mes', 'SemanaDelAno']
        X_train = datos_entrenamiento[features]
        y_train = datos_entrenamiento['Large Bags']
        X_impute = datos_imputar[features]

        # Escalar las características (MUY importante para KNN)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_impute_scaled = scaler.transform(X_impute) # Solo transformar con el scaler ajustado a train

        # Crear y entrenar el modelo KNeighborsRegressor
        # weights='distance' -> vecinos más cercanos tienen más influencia
        knn_reg = KNeighborsRegressor(n_neighbors=n_neighbors_knn, weights='distance')
        knn_reg.fit(X_train_scaled, y_train)

        # Predecir los valores para los ceros
        predicciones = knn_reg.predict(X_impute_scaled)

        # Ajustar predicciones negativas a 0
        predicciones_ajustadas = np.maximum(0, predicciones) # predict devuelve numpy array

        # Obtener índices originales para actualizar df2
        indices_imputar = datos_imputar.index

        # *** NUEVO: Guardar la predicción ajustada en la nueva columna ***
        df2.loc[indices_imputar, COLUMNA_PREDICCION] = predicciones_ajustadas
        indices_imputados_global.update(indices_imputar) # Actualizar conjunto de índices donde se hizo predicción

        # --- NO SOBRESCRIBIR 'Large Bags' ---
        # Ahora que guardamos la predicción, no necesitamos (ni queremos)
        # sobrescribir la columna original 'Large Bags' aquí.
        # El siguiente script usará 'Large Bags' y 'KNN_Prediction' para crear 'Large Bags Filled'.
        # df2.loc[indices_imputar, 'Large Bags'] = predicciones_ajustadas # <-- COMENTADO/ELIMINADO

        # --- NO ACTUALIZAR TOTALES AQUÍ ---
        # Es más lógico actualizar los totales cuando se crea 'Large Bags Filled'
        # if 'Total Bags' in df2.columns: df2.loc[indices_imputar, 'Total Bags'] += predicciones_ajustadas # <-- COMENTADO
        # if 'Total Volume' in df2.columns: df2.loc[indices_imputar, 'Total Volume'] += predicciones_ajustadas # <-- COMENTADO

        print(f" Predicciones guardadas para {len(indices_imputar)} puntos.")

    except Exception as e:
        print(f" ERROR ({type(e).__name__}: {str(e)[:100]}...).")
        # Considerar añadir 'continue' si quieres que el bucle siga con el siguiente grupo tras un error

    finally:
         # Limpieza de Memoria al final de CADA iteración
         variables_a_borrar = ['group_df', 'datos_entrenamiento', 'datos_imputar',
                               'X_train', 'y_train', 'X_impute', 'X_train_scaled',
                               'X_impute_scaled', 'knn_reg', 'scaler',
                               'predicciones', 'predicciones_ajustadas']
         for var_name in variables_a_borrar:
              # Usar try-except por si alguna variable no llegó a definirse antes de un error
              try:
                  if var_name in locals(): del locals()[var_name]
              except NameError: pass # Ignorar si la variable no existe
         # Llamar al recolector de basura explícitamente
         # collected = gc.collect() # Descomentar si realmente tienes problemas graves de memoria

print(f"\n--- Proceso de generación de predicciones KNN en df2 finalizado ---")
print(f"Se generaron predicciones para {len(indices_imputados_global)} puntos en total.")
print(f"La columna '{COLUMNA_PREDICCION}' ahora contiene las predicciones (o NaN si no se aplicó).")
print("La columna 'Large Bags' original NO ha sido modificada.")

# --- Limpieza Opcional: Eliminar columnas auxiliares de fecha ---
# Considera si las necesitarás más adelante antes de borrarlas
# df2 = df2.drop(columns=['Fecha_Ordinal', 'Mes', 'SemanaDelAno'], errors='ignore')
# print("Columnas auxiliares de fecha eliminadas.")

Usando el DataFrame 'df2' existente.

Preparando datos en df2 (índice, tipos numéricos, features de tiempo)...
Asegurando que ['Large Bags', 'Total Bags', 'Total Volume', 'AveragePrice'] sean numéricas y rellenando NaNs con 0...
Optimizando tipos de datos...
Optimización de tipos completada.

--- Iniciando proceso de imputación para 'Large Bags' con KNeighborsRegressor en df2 ---
--- Las predicciones se guardarán en la columna 'KNN_Prediction' ---
Umbral mínimo de muestras de entrenamiento (>0): 10
Número de vecinos (k) para KNN: 5
Procesando Grupo 1/108: conventional/Albany...  OK (No hay ceros que imputar).
Procesando Grupo 2/108: conventional/Atlanta...  OK (No hay ceros que imputar).
Procesando Grupo 3/108: conventional/BaltimoreWashington...  OK (No hay ceros que imputar).
Procesando Grupo 4/108: conventional/Boise...  Entrenando con 142, prediciendo para 27 ceros...  ERROR (ValueError: Must have equal len keys and value when setting with an iterable...).
Procesando Grupo 5/108: c

<ipython-input-5-862c06b336d1>:111: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_grupos = len(df2.groupby(['type', 'region']))
<ipython-input-5-862c06b336d1>:113: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (tipo, region), group_df in df2.groupby(['type', 'region']):


 ERROR (ValueError: Must have equal len keys and value when setting with an iterable...).
Procesando Grupo 73/108: organic/Houston...  Entrenando con 103, prediciendo para 66 ceros...  ERROR (ValueError: Must have equal len keys and value when setting with an iterable...).
Procesando Grupo 74/108: organic/Indianapolis...  OK (No hay ceros que imputar).
Procesando Grupo 75/108: organic/Jacksonville...  Entrenando con 88, prediciendo para 81 ceros...  ERROR (ValueError: Must have equal len keys and value when setting with an iterable...).
Procesando Grupo 76/108: organic/LasVegas...  Entrenando con 147, prediciendo para 22 ceros...  ERROR (ValueError: Must have equal len keys and value when setting with an iterable...).
Procesando Grupo 77/108: organic/LosAngeles...  Entrenando con 134, prediciendo para 35 ceros...  ERROR (ValueError: Must have equal len keys and value when setting with an iterable...).
Procesando Grupo 78/108: organic/Louisville...  Entrenando con 167, prediciendo para 

In [6]:
"""
# --- BLOQUE DE CÓDIGO DE GRAFICACIÓN (ADAPTADO PARA KNN) ---

# Importar KNN y Scaler para re-entrenar
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
import matplotlib.dates as mdates


# --- Selección de Grupos y Generación de Gráficos ---
grupos_a_graficar = [
    ('conventional', 'BuffaloRochester'),
    ('organic', 'NewYork'),
    ('conventional', 'SanFrancisco'),
    # ('conventional', 'Nashville'),
    # ('conventional', 'West')
]

# Verificar si las variables necesarias existen
if ('indices_imputados_global' not in locals() or 'df2' not in locals()
    or 'min_train_samples' not in locals() or 'n_neighbors_knn' not in locals()):
     print("Error: Variables de imputación no encontradas (df2, indices_imputados_global, etc.).")
     print("Asegúrate de ejecutar el bloque de imputación KNN primero.")
     exit()
else:
    print(f"\n--- Generando gráficos para {len(grupos_a_graficar)} grupos seleccionados usando KNN (datos de df2) ---")

    # Re-asegurar columnas necesarias para graficar
    if not all(col in df2.columns for col in ['type', 'region', 'Date', 'Large Bags', 'Fecha_Ordinal', 'Mes', 'SemanaDelAno']):
         print("Error: Faltan columnas necesarias para graficar en df2."); exit()

    for tipo_plot, region_plot in grupos_a_graficar:

        print(f"\nGenerando gráfico para: type='{tipo_plot}', region='{region_plot}'")

        df_grupo = df2[(df2['type'] == tipo_plot) & (df2['region'] == region_plot)].sort_index().copy()

        if df_grupo.empty: print("   -> No hay datos para este grupo."); continue

        indices_imputados_grupo = df_grupo.index.intersection(indices_imputados_global)
        mask_imputed = df_grupo.index.isin(indices_imputados_grupo)
        mask_original_nonzero = (df_grupo['Large Bags'] > 0)
        mask_original_zero_not_imputed = (~mask_original_nonzero) & (~mask_imputed)

        # --- Re-entrenar el modelo KNN y predecir para todo el grupo (para la línea) ---
        train_data_plot = df_grupo[mask_original_nonzero]

        knn_reg_plot = None
        pred_line = None
        pred_dates = None
        can_plot_line = False

        # Comprobar si hay suficientes datos originales (>0) y si son suficientes para los vecinos k
        if len(train_data_plot) >= min_train_samples and len(train_data_plot) >= n_neighbors_knn:
            print(f"   -> Re-entrenando modelo KNN con {len(train_data_plot)} puntos originales (>0).")
            try:
                features = ['Fecha_Ordinal', 'Mes', 'SemanaDelAno']
                X_train_plot = train_data_plot[features]
                y_train_plot = train_data_plot['Large Bags']
                # Preparar features para TODOS los puntos del grupo para obtener la línea de predicción
                X_group_plot = df_grupo[features]

                # Escalar
                scaler_plot = StandardScaler()
                X_train_plot_scaled = scaler_plot.fit_transform(X_train_plot)
                X_group_plot_scaled = scaler_plot.transform(X_group_plot) # Usar mismo scaler

                # Entrenar
                knn_reg_plot = KNeighborsRegressor(n_neighbors=n_neighbors_knn, weights='distance')
                knn_reg_plot.fit(X_train_plot_scaled, y_train_plot)

                # Predecir para todos los puntos del grupo
                pred_line_raw = knn_reg_plot.predict(X_group_plot_scaled)
                pred_line = np.maximum(0, pred_line_raw) # Cap negativos
                pred_dates = df_grupo.index # Fechas correspondientes
                can_plot_line = True

            except Exception as e:
                print(f"   -> Error al re-entrenar o predecir para la línea KNN: {e}")
        else:
            print(f"   -> No se puede trazar la línea KNN (datos originales > 0 son {len(train_data_plot)}, se necesitan {max(min_train_samples, n_neighbors_knn)}).")

        # --- Crear el Gráfico ---
        plt.figure(figsize=(15, 7))

        plt.scatter(df_grupo.loc[mask_original_nonzero].index, df_grupo.loc[mask_original_nonzero, 'Large Bags'],
                    label='Original (>0)', alpha=0.6, color='blue', s=30)
        if not df_grupo.loc[mask_imputed].empty:
             plt.scatter(df_grupo.loc[mask_imputed].index, df_grupo.loc[mask_imputed, 'Large Bags'],
                         label='Imputado (Originalmente 0)', alpha=0.9, color='orange', marker='X', s=60)
        if not df_grupo.loc[mask_original_zero_not_imputed].empty:
              plt.scatter(df_grupo.loc[mask_original_zero_not_imputed].index, df_grupo.loc[mask_original_zero_not_imputed, 'Large Bags'],
                         label='Originalmente 0 (No Imputado)', alpha=0.7, color='red', marker='o', s=40)

        # Graficar la línea de predicción KNN si se pudo calcular
        if can_plot_line and pred_line is not None and pred_dates is not None:
             # Crear un df temporal para ordenar por fecha antes de graficar linea
             line_df = pd.DataFrame({'Date': pred_dates, 'Prediction': pred_line}).sort_values(by='Date')
             plt.plot(line_df['Date'].dt.to_pydatetime(), line_df['Prediction'],
                      color='green', linestyle='--', linewidth=2, label=f'Predicción KNN (k={n_neighbors_knn})')

        plt.title(f'Large Bags vs Tiempo (Imputado con KNN) - {tipo_plot.capitalize()} / {region_plot}')
        plt.xlabel('Fecha')
        plt.ylabel('Large Bags')
        plt.legend()
        plt.grid(True, axis='y', linestyle=':')
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator(minticks=5, maxticks=10))
        plt.gcf().autofmt_xdate()
        plt.tight_layout()
        plt.show()
    """

'\n# --- BLOQUE DE CÓDIGO DE GRAFICACIÓN (ADAPTADO PARA KNN) ---\n\n# Importar KNN y Scaler para re-entrenar\nfrom sklearn.neighbors import KNeighborsRegressor\nfrom sklearn.preprocessing import StandardScaler\nimport matplotlib.dates as mdates\n\n\n# --- Selección de Grupos y Generación de Gráficos ---\ngrupos_a_graficar = [\n    (\'conventional\', \'BuffaloRochester\'),\n    (\'organic\', \'NewYork\'),\n    (\'conventional\', \'SanFrancisco\'),\n    # (\'conventional\', \'Nashville\'),\n    # (\'conventional\', \'West\')\n]\n\n# Verificar si las variables necesarias existen\nif (\'indices_imputados_global\' not in locals() or \'df2\' not in locals()\n    or \'min_train_samples\' not in locals() or \'n_neighbors_knn\' not in locals()):\n     print("Error: Variables de imputación no encontradas (df2, indices_imputados_global, etc.).")\n     print("Asegúrate de ejecutar el bloque de imputación KNN primero.")\n     exit()\nelse:\n    print(f"\n--- Generando gráficos para {len(grupos_a_gr